# Telco Customer Churn

link: https://www.kaggle.com/datasets/blastchar/telco-customer-churn

## Bibliotecas 

In [1]:
# manipulação de dados
import pandas as pd
import numpy as np

# pre-processamento
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

# separação treino e teste
from sklearn.model_selection import train_test_split

# modelos
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# metricas
from sklearn.metrics import accuracy_score, confusion_matrix

## Leitura dos dados 

In [2]:
df_raw = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df_raw.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## Verificação dos dados

### tamanho dos dados

In [3]:
print('Total de linhas: ', df_raw.shape[0])
print('Total de colunas: ', df_raw.shape[1])

Total de linhas:  7043
Total de colunas:  21


### vertificar dados faltantes

In [4]:
df_raw.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

### verificar dados duplicados

In [5]:
df_raw.duplicated().sum()

0

### tipos dos dados

In [6]:
df_raw.dtypes

customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

## Seleção de features

**Não precisando do customerID**

In [7]:
df = df_raw.copy()
# não precisamos do customerID
df = df.drop('customerID', axis = 1)
#df = df.dropna(subset=['TotalCharges'])

# a variavel TotalCharges está como categorico
df['TotalCharges'] = df['TotalCharges'].replace(' ', 0).astype(float)


In [8]:
# target
#y = df['Churn']
y = pd.get_dummies(df['Churn'], drop_first = True)
y['Target'] = y['Yes']
y.drop('Yes', axis = 1, inplace=True)

# x categorico
onehotencoder = OneHotEncoder()
variaveis_categoricas = df.select_dtypes(include=['object']).drop('Churn', axis=1)
x_categorico = onehotencoder.fit_transform(variaveis_categoricas).toarray()
x_categorico = pd.DataFrame(x_categorico)

# x numerico
variaveis_numericas = df.select_dtypes(include=['int','float'])
x_numerico = variaveis_numericas

# variavel categorica
x_all = pd.concat([x_categorico, x_numerico], axis=1)
scaler = MinMaxScaler()
x = scaler.fit_transform(x_all)
x


C:\Users\Ben-Hur\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\Ben-Hur\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


array([[1.        , 0.        , 0.        , ..., 0.01388889, 0.11542289,
        0.00343704],
       [0.        , 1.        , 1.        , ..., 0.47222222, 0.38507463,
        0.21756402],
       [0.        , 1.        , 1.        , ..., 0.02777778, 0.35422886,
        0.01245279],
       ...,
       [1.        , 0.        , 0.        , ..., 0.15277778, 0.11293532,
        0.03989153],
       [0.        , 1.        , 0.        , ..., 0.05555556, 0.55870647,
        0.03530306],
       [0.        , 1.        , 1.        , ..., 0.91666667, 0.86965174,
        0.78810105]])

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.25)

In [10]:
x_train.shape

(5282, 45)

In [11]:
x_test.shape

(1761, 45)

In [12]:
y_train.shape

(5282, 1)

In [13]:
y_test.shape

(1761, 1)

## Regressão logistica

In [14]:
logistic_model = LogisticRegression()
logistic_model.fit(x_train, y_train)

C:\Users\Ben-Hur\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Ben-Hur\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [15]:
y_pred = logistic_model.predict(x_test)
y_pred

array([1, 1, 0, ..., 0, 0, 0], dtype=uint8)

In [16]:
accuracy_score(y_test, y_pred)

0.8069278818852924

In [17]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[1172,  137],
       [ 203,  249]], dtype=int64)

## Arvore de decisão

In [43]:
%%time
tree_model = DecisionTreeClassifier(max_depth=2,
                                   min_samples_leaf = 5)
tree_model.fit(x_train, y_train)

Wall time: 8.01 ms


DecisionTreeClassifier(max_depth=2, min_samples_leaf=5)

In [44]:
y_pred = tree_model.predict(x_test)
y_pred

array([1, 1, 0, ..., 0, 0, 0], dtype=uint8)

In [23]:
accuracy1 = accuracy_score(y_test, y_pred)

0.7637705848949461

In [45]:
accuracy2 = accuracy_score(y_test, y_pred)

In [37]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[1074,  235],
       [ 181,  271]], dtype=int64)

In [29]:
def treina_arvore_decisao(x_train, y_train, x_test, y_test, max_d, min_s):
    modelo = DecisionTreeClassifier(max_depth = max_d,
                                   min_samples_leaf = min_s)
    modelo.fit(x_train, y_train)
    
    y_pred = modelo.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    return modelo, accuracy

In [38]:
%%time
modelo_list = list()
max_d = list()
min_s = list()
accuracy_list = list()

for d in range(1,10):
    for s in range(1 ,31):
        print('Treinando modelo', s)
        max_d.append(d)
        min_s.append(s)
        modelo, accuracy = treina_arvore_decisao(x_train, y_train, x_test, y_test, d, s)
        
        modelo_list.append(modelo)
        accuracy_list.append(accuracy)
        
busca_ad = pd.DataFrame(list(zip(max_d, min_s, accuracy_list, modelo_list)),
                       columns = ['Max Deph', 'Min Sample Leaf', 'Accuracy', 'Modelo'])
#busca_ad

Treinando modelo 1
Treinando modelo 2
Treinando modelo 3
Treinando modelo 4
Treinando modelo 5
Treinando modelo 6
Treinando modelo 7
Treinando modelo 8
Treinando modelo 9
Treinando modelo 10
Treinando modelo 11
Treinando modelo 12
Treinando modelo 13
Treinando modelo 14
Treinando modelo 15
Treinando modelo 16
Treinando modelo 17
Treinando modelo 18
Treinando modelo 19
Treinando modelo 20
Treinando modelo 21
Treinando modelo 22
Treinando modelo 23
Treinando modelo 24
Treinando modelo 25
Treinando modelo 26
Treinando modelo 27
Treinando modelo 28
Treinando modelo 29
Treinando modelo 30
Treinando modelo 1
Treinando modelo 2
Treinando modelo 3
Treinando modelo 4
Treinando modelo 5
Treinando modelo 6
Treinando modelo 7
Treinando modelo 8
Treinando modelo 9
Treinando modelo 10
Treinando modelo 11
Treinando modelo 12
Treinando modelo 13
Treinando modelo 14
Treinando modelo 15
Treinando modelo 16
Treinando modelo 17
Treinando modelo 18
Treinando modelo 19
Treinando modelo 20
Treinando modelo 2

In [39]:
busca_ad.sort_values('Accuracy', ascending=False)

,Max Deph,Min Sample Leaf,Accuracy,Modelo
109,4,20,0.797274,"DecisionTreeClassifier(max_depth=4, min_sample..."
119,4,30,0.797274,"DecisionTreeClassifier(max_depth=4, min_sample..."
118,4,29,0.797274,"DecisionTreeClassifier(max_depth=4, min_sample..."
117,4,28,0.797274,"DecisionTreeClassifier(max_depth=4, min_sample..."
116,4,27,0.797274,"DecisionTreeClassifier(max_depth=4, min_sample..."
...,...,...,...,...
15,1,16,0.743328,"DecisionTreeClassifier(max_depth=1, min_sample..."
14,1,15,0.743328,"DecisionTreeClassifier(max_depth=1, min_sample..."
13,1,14,0.743328,"DecisionTreeClassifier(max_depth=1, min_sample..."
12,1,13,0.743328,"DecisionTreeClassifier(max_depth=1, min_sample..."


In [41]:
modelo_selecionado = busca_ad.iloc[109]['Modelo']

## Redes Neurais